In [28]:
%load_ext autoreload
%autoreload 2

from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import time
import uuid
from modules.db_operations import DatabaseConnector
from modules.sends import SendMail
EMAIL_ADDRESS_FROM = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet
# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

#Read in file
pd.set_option('display.max_columns', None)
df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\SQL_Scripts\email_prospects_csvs\baseball.csv')
df = df.drop_duplicates(subset='email')
df = df.reset_index(drop = True)
# df = df.iloc[751:800]

def process(df, contact_column, sport, email_campaign_name, global_subject_line):

    data_list = []
    processed_emails = set()

    #Limit df itterrows to test
    for index, row in df.iterrows():

        # Get the current date
 
        #Set the time zone to Central Time
        central_time_zone = pytz.timezone('America/Chicago')
        now_central = datetime.now(central_time_zone)
        formatted_date = now_central.strftime("%m%d%Y%H%M%S")
        #When running campaigns to non schools
        try:
            school = row['School Name']
        except:
            school = None

        # png_path = row['PNG_PATH']
        contact = row[contact_column]
         #estiablish unique identifier, in order to match up FK relation, and merges
        unique_identifier = str(uuid.uuid4())

        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date, unique_identifier]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)
        
        SendMail.send(EMAIL_ADDRESS_FROM, EMAIL_PASS, global_subject_line, school, sport, contact, unique_identifier)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent', 'recipient_id'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date_sent'] = data_list['date_sent'].astype(str)

    data_list['subject'] = global_subject_line
    data_list['contact_position'] = contact_column
    data_list['sent_from'] = EMAIL_ADDRESS_FROM
    data_list['email_campaign_tag'] = email_campaign_name
    data_list['date_sent'] = now_central.strftime("%Y-%m-%d %H:%M:%S")

    data_list.to_csv('output.csv', index=False)

    return(data_list)

# --------------------------------------Running Process--------------------------

#Temporary testing to modify the Athletic Director email to my own
# df.loc[0, 'email'] = '2015samtaylor@gmail.com'
# df.loc[1, 'email'] = 'sammytaylor2006@yahoo.com'
# df.loc[2, 'email'] = 'samuel.taylor@greendot.org'
# df.loc[3, 'email'] = 'sammtaylor2006@gmail.com'
# df.loc[3, 'email'] = 'jerrybons2006@gmail.com'

# #Must be the column name of the position wanting to send, sport fills in email headers, third arg is campaign name
# SCHOOL INTRO (contact_column, sport, email_campaign_name, global_subject_line)
#First arg is relevant for School Intro
#Second arg is for sport intro
#Third arg is for email campaign tag in the db
# Fourth arg is the Global subject line declared above. 

#Subject line references format string in sends.py, and is also the subject line of the email
# global global_subject_line
# global_subject_line = 'Official MLB Jerseys in Stock!!'

# email_history = process(df, 'email' ,'Baseball', 'Official MLB Jerseys in Stock!!', global_subject_line)

# # Instantiate the DatabaseConnector class, and send over the dataframe via append
# db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
# db_connector.send(email_history)

In [ ]:
#Need to make a new SMTP connection after 50 emails. 
#Sends 50 emails and writes to the DB in 1 min roughly 


#Need to pull from the DB and see what was the last email sent to. 
#Then start +1 from that point and send 50 with a new SMTP connection. 

#output gets re-written everytime. This is the csv that is written to the db

#When the db is written to they all come across with the same date_sent time. 
#Therefore I can not start on the one that was last left, unless done on output.csv before it gets overwritten. 
#How would this far for starting at the beginning? 

#Query in most recent records from db?


#On the first loop initially start at 50 emails. 
#Sleep for 45 seconds, and then re-instantiate a connection


#Need to log out errors for what is sent or append to csv
#Log out the amount of records to the DB


In [27]:
#gets the next 50 based on the email_history variable returned from the processs function. 
# df = SendMail.get_next_50(email_history)

# email_history = process(df, 'email' ,'Baseball', 'Official MLB Jerseys in Stock!!', global_subject_line)
# db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
# db_connector.send(email_history)

In [30]:
global global_subject_line
global_subject_line = 'Official MLB Jerseys in Stock!!'


# Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
while len(df) >= 1:

    df = SendMail.get_next_50(email_history)

    new_point = df.index[0]
    end_point = df.index[-1]   
    print(new_point, end_point) 
    
    # Process the emails
    email_history = process(df, 'email' ,'Baseball', 'Official MLB Jerseys in Stock!!', global_subject_line)

    # Send the processed emails to the database
    db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
    db_connector.send(email_history)

    interval_seconds = 60
    # Wait for the specified interval before the next iteration
    time.sleep(interval_seconds)


#Sends 500 emails in 15 mins
#Limit to 1500 emails per day
#Anything over 2000 the entire account gets locked for 24 hours. 
#Limits to 2000 emails per day
# SMTPDataError: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 w4-20020a4ae9e4000000b005914f455774sm848190ooc.34 - gsmtp')

1950 1999


SMTPDataError: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 p9-20020a056830130900b006dbab84fa35sm787873otq.43 - gsmtp')